In [1]:
!pip install pykrx
!pip install -U finance-datareader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.2/254.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 1.8 MB/s eta 0:00:00


In [2]:
from pykrx import stock
import FinanceDataReader as fdr
import pandas as pd

In [5]:
ETF_df = fdr.StockListing('ETF/KR')
ETF = ETF_df[ETF_df['Symbol'].isin(['305540', '305720', '364980'])]
ETF

,Symbol,Category,Name,Price,RiseFall,Change,ChangeRate,NAV,EarningRate,Volume,Amount,MarCap
36,305540,2,TIGER 2차전지테마,16980,2,220,1.31,17004.0,-10.7562,395452,6606,9780
38,305720,2,KODEX 2차전지산업,13835,2,200,1.47,13849.0,-15.2316,685311,9324,9671
103,364980,2,TIGER 2차전지TOP10,9390,2,135,1.46,9395.0,-10.9240,1694899,15740,3296


In [15]:
ETF_close = pd.DataFrame()
for idx, values in ETF.iterrows():
  df = fdr.DataReader(values['Symbol'], '2018-01-01')[['Close']].rename(columns = {'Close' : values['Name']}) # 2021년 1월 1일부터 현재까지
  ETF_close = pd.concat([ETF_close, df], axis = 1)
ETF_close.dropna(inplace = True)

In [20]:
ETF_norm = ETF_close / ETF_close.iloc[0] * 100

import plotly.graph_objects as go

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=ETF_norm.index, y=ETF_norm['TIGER 2차전지테마'],
                    mode='lines',
                    name='TIGER 2차전지테마'))
fig.add_trace(go.Scatter(x=ETF_norm.index, y=ETF_norm['KODEX 2차전지산업'],
                    mode='lines',
                    name='KODEX 2차전지산업'))
fig.add_trace(go.Scatter(x=ETF_norm.index, y=ETF_norm['TIGER 2차전지TOP10'],
                    mode='lines', name='TIGER 2차전지TOP10'))

fig.show()

### 종목 구성 비교

In [27]:
etf_portfolio = pd.DataFrame()
for idx, values in ETF.iterrows():
  df = stock.get_etf_portfolio_deposit_file(values['Symbol'])[['비중']].rename(columns = {'비중' : values['Name']})
  etf_portfolio = pd.concat([etf_portfolio, df], axis = 1)
etf_portfolio.head()

,TIGER 2차전지테마,KODEX 2차전지산업,TIGER 2차전지TOP10
티커,,,
373220,19.16,9.540000,29.059999
051910,15.00,11.860000,7.820000
005490,14.51,19.540001,22.980000
006400,13.10,10.160000,20.670000
003670,7.00,7.230000,3.040000


In [33]:
etf_portfolio.reset_index().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   티커               31 non-null     object 
 1   TIGER 2차전지테마     30 non-null     float32
 2   KODEX 2차전지산업     26 non-null     float32
 3   TIGER 2차전지TOP10  11 non-null     float32
dtypes: float32(3), object(1)
memory usage: 748.0+ bytes


In [36]:
kr_stock_list = fdr.StockListing('KRX')
kr_stock_list.head()

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,56300,2,-400,-0.71,56500,57500,55900,31429382,1780686972300,337292714075000,5969782550,STK
1,000660,KR7000660001,SK하이닉스,KOSPI,,170600,2,-1100,-0.64,170400,172900,168100,4039812,688259294300,124488404415000,728002365,STK
2,373220,KR7373220003,LG에너지솔루션,KOSPI,,393500,1,10000,2.61,384500,395500,380500,290555,113219937500,92430000000000,234000000,STK
3,207940,KR7207940008,삼성바이오로직스,KOSPI,,919000,1,3000,0.33,917000,937000,917000,74323,68766176000,65551254000000,71174000,STK
4,005380,KR7005380001,현대차,KOSPI,,215500,2,-1500,-0.69,217000,219000,213000,521041,112476852000,45338605351500,209416191,STK


In [41]:
ETF_ratio = pd.merge(etf_portfolio.reset_index(), kr_stock_list[['Code', 'Name']], how = 'left', left_on = '티커', right_on = 'Code').drop(['티커', 'Code'], axis = 1).set_index('Name')
ETF_ratio

,TIGER 2차전지테마,KODEX 2차전지산업,TIGER 2차전지TOP10
Name,,,
LG에너지솔루션,19.16,9.540000,29.059999
LG화학,15.00,11.860000,7.820000
POSCO홀딩스,14.51,19.540001,22.980000
삼성SDI,13.10,10.160000,20.670000
포스코퓨처엠,7.00,7.230000,3.040000
에코프로비엠,6.78,8.730000,5.050000
SK이노베이션,5.06,8.990000,4.290000
에코프로,4.56,4.610000,4.310000
에코프로머티,2.94,1.090000,1.230000


In [45]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

labels = ETF_ratio.index

fig = make_subplots(1, 3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=ETF_ratio.columns)
fig.add_trace(go.Pie(labels=labels, values=ETF_ratio.iloc[:, 0], scalegroup='one',
                     name=ETF_ratio.columns[0]), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=ETF_ratio.iloc[:, 1], scalegroup='one',
                     name=ETF_ratio.columns[1]), 1, 2)
fig.add_trace(go.Pie(labels=labels, values=ETF_ratio.iloc[:, 2], scalegroup='one',
                     name=ETF_ratio.columns[2]), 1, 3)

fig.update_layout(title_text='2차전지 ETF 비교')
fig.show()

In [47]:
ETF_ratio.corr()

,TIGER 2차전지테마,KODEX 2차전지산업,TIGER 2차전지TOP10
TIGER 2차전지테마,1.000000,0.872333,0.890226
KODEX 2차전지산업,0.872333,1.000000,0.697256
TIGER 2차전지TOP10,0.890226,0.697256,1.000000


### [실습] 아래 3가지 ETF 비교하기
- TIGER 코스피고배당
- PLUS 고배당주
- RISE 고배당
